## Fluídos Conformes

Código para testear la estabilidad alrededor de una solución de equilibrio.

Las ecuaciones son:

$(\zeta^a, \zeta^{ab})$ -> $f^j = (\mu, v, x_i)$ -> $(-1,0,0,0,0)$

$$
\partial_t c^j = \partial_i F^{ij}(f) + I^j(f) = \frac{\partial F^{ij}}{\partial f^k}\partial_i f^k
$$

$$
c^j = C^j(f) = (e, S, c_i)
$$

Lo que hacemos es tomar el jacobiano del flujo,

$\frac{\partial F^i}{\partial f^j}$ donde $f^j$ son las variables no conservativas para las cuales tenemos las expresiones del Flujo. 
A estas las multiplicamos por $[\frac{\partial c^k}{\partial f^j}]^{-1}$.
Tenemos así 

$$
P^{il}{}_j := \frac{\partial F^{il}}{\partial c^j} = \frac{\partial F^{il}}{\partial f^k}\frac{\partial f^k}{\partial c^j}.
$$

Evaluamos todo en el vector $(-1,0,0,0,0)$ o sea estamos en el frame con velocidad cero y todas las variables fuera de equilibrio también cero.

Lo mismo hacemos con la fuente. 

$$
II^i{}_j = \frac{\partial I^i}{\partial f^k} \frac{\partial f^k}{\partial c^j}
$$

Tenemos así el sistema linearizado en el punto de equilibrio en las variables conservativas. Multiplicamos $P^i{}_j$ por un número complejo cualquiera y para todos ellos los autovalores debieran tener parte real negativa o nula. Pero no es así...


Alternativamente podemos diferenciar $c^i$ con respecto a los $f^j$ para obtener,

$$
\partial_t c^j = \frac{\partial c^j}{\partial f^i} \partial_t f^i
$$

y por lo tanto,

$$
\partial_t f^i = \frac{\partial c^j}{\partial f^i}^{-1} \frac{\partial F^{lj}}{\partial f^k}\partial_l f^k := \tilde{P}^i{}_k\partial_l f^k
$$

A las que simplemente sumamos el término $\frac{\partial c^j}{\partial f^i}^{-1}\frac{\partial I^j}{\partial f^k}$ para tener el sistema linearizado con respecto a las $f^i$.

In [196]:
using Symbolics
using LinearAlgebra
using Latexify

In [4]:
function Flux(flu,par)
    χ = par
    #flux = zeros(5)
    μ = flu[1]  
    T = (abs(μ))^(-1//2) 
    v = flu[2]
    x1 = flu[3]
    x2 = flu[4]
    x3 = flu[5]
    χ₀ = χ[1]
    χ₁ = χ[2]
    χ₂ = χ[3]
    γ = (1 - v^2)^(-1//2)
    τ = 2χ₁ * x3 * T / (γ*μ^3) + 24χ₂*(1//2*(1-v^2)x3^2 + 14//3 * x2^2 + 7/5*(1-v^2)x1*x3)/μ^5
    ρ = -6χ₀ / μ^2 - 6χ₁*x1/(γ * μ^4 * T) + 42χ₂*(6//5 *x1^2 + 10γ^2*x2^2 + 3//2*(v^2-1)^2*x3^2)/(μ^5 * γ^2)
#    Q = 10χ₀ * x2 * T / μ^3 + 168χ₂ * x2 * (x1 - (v^2 - 1)x3)/(γ * μ^5)
    Q = 10χ₁ * x2 * T / μ^3 + 168χ₂ * x2 * (x1 - (v^2 - 1)x3)/(γ * μ^5)
    #flux[1] = 4//3 * ρ*γ^2*v + γ*Q*(1+v^2)+ τ*v
    #flux[2] = 4//3 * ρ*(γ^2*v^2 + 1//4) + 2v*γ*Q + τ
    #flux[3] = χ₁*γ*v*(6*γ^2 - 1)/μ^3/T - 12χ₂*(v*(6γ^2 - 1)*x1 + (6γ^2*(2v^2 + 1)-1)*x2 + v*(v^2+2)*x3)/μ^4
    #flux[4] = χ₁*γ*(6γ^2*v^2+1)/μ^3/T - 12χ₂*((6γ^2*v+1)*x1 + v*(6γ^2*(1+v^2)+1)*x2 + (2v^2+1)*x3)/μ^4
    #flux[5] = 3χ₁*γ*v*(2γ^2*v^2+1)/μ^3/T - 12χ₂*(v*(6γ^2*v^2+3)*x1 +3*(1+6γ^2*v^2)*x2 + 3v*x3)/μ^4
    #return -flux[:]
    return [4//3 * ρ*γ^2*v + γ*Q*(1+v^2)+ τ*v; 
            4//3 * ρ*(γ^2*v^2 + 1//4) + 2v*γ*Q + τ;
            χ₁*γ*v*(6*γ^2 - 1)/μ^3/T   - 12χ₂*(v*(6γ^2 - 1)*x1   + (6γ^2*(2v^2 + 1)-1)*x2 + v*(v^2+2)*x3)/μ^4;
            χ₁*γ*(6γ^2*v^2+1)/μ^3/T    - 12χ₂*((6γ^2*v+1)*x1     + v*(6γ^2*(1+v^2)+1)*x2  + (2v^2+1)*x3)/μ^4;
            3χ₁*γ*v*(2γ^2*v^2+1)/μ^3/T - 12χ₂*(v*(6γ^2*v^2+3)*x1 + 3*(1+6γ^2*v^2)*x2      + 3v*x3)/μ^4]
end

Flux (generic function with 1 method)

Chequeamos con los valores de Marcelo. 

In [5]:
χ = [- 1.; - 2.; - 10.]
flu = [- 5.; 0.5; 2.1; 0.5; 5.1]
Flu = [5.921317694643806; 6.02302807825841; 3.713391378258412;  4.136735467078638; 3.444079555898864]

Flux(flu,χ) - Flu;

Calculamos el Jacobiano

In [6]:
@variables f[1:5], c[1:5], p[1:3]#, fl[1:5]

JFS = Symbolics.jacobian(Flux(f,p),f);
JF_exp = Symbolics.build_function(JFS, f, p);
JFac = eval(JF_exp[1]);

Definimos algunos valores para testear

In [7]:
@variables x1, x2, x3

flu=[-1.;0;0;0;0];
con=ones(5)
χ=[-1.,-0.5,-5.]

3-element Vector{Float64}:
 -1.0
 -0.5
 -5.0

El Jacobiano en equilibrio.

In [8]:
JFac(flu,p)

5×5 Matrix{Num}:
  0.0    (-8//1)*p₁          0.0    -10.0p₂      0.0
 -4.0p₁           0.0       -2.0p₂    0.0       -2.0p₂
  0.0            -5.0p₂     -0.0    -60.0p₃     -0.0
 -2.5p₂           0.0    -12p₃       -0.0    -12p₃
  0.0            -3.0p₂     -0.0    -36.0p₃     -0.0

Ahora traemos el Jacobiano de la transformación entre las variables fluido/conservativas.

In [9]:
include("inversion_ext.jl");

Para posterior uso calculamos las variables conservativas correspondientes al estado de equilibrio. Esto no me lo esperaba, pensaba que me darían c1=c2=c3=0!

In [11]:
con = F(flu,zeros(5), p);

Jac(flu,con,p)

In [12]:
P(flu,c,p) = Jac(flu,c,p) \ JFac(flu,p)

P (generic function with 1 method)

In [13]:
flu = [-1;0.9;0;0;0]
M0=simplify.(substitute.(P(flu,c,p), (Dict(p[2] => 0),)))

5×5 Matrix{Num}:
  0.821918   0.913242   -2.32481e-15    2.30811e-14  -1.6297e-18
  0.024726   0.821918   -0.271687       2.69735      -0.000190454
 -0.0       -0.0        -5.29874       63.8535        0.00194622
 -0.0       -0.0         2.62439      -26.0554        0.00183971
 -0.0       -0.0       -15.5246       167.936         0.955344

In [68]:
P(flu,c,p);

LoadError: syntax: incomplete: premature end of input

In [15]:
18.75/3.75*36

180.0

In [16]:
P(flu,c,p)[4,3];

In [17]:
P(flu,c,p)';

In [67]:
Ratio = P(flu,c,p)'[4,1]/P(flu,c,p)'[4,3];

In [19]:
#par_f = [-1,-0.5,-10]
#substitute(P(flu,c,p)'[4,1]/P(flu,c,p)'[4,3], (Dict(p => par_f),))


In [20]:
simplify(12*(10p[1]*(-36p[3] - 3.75p[2]^2/p[1]) + 6p[2]*(6.25p[2] - 60p[3])) / (6.25p[2] - 60p[3]) / p[1])

(p₁^-1)*((6.25p₂ - (60p₃))^-1)*(120p₁*(-36p₃ - (3.75(p₁^-1)*(p₂^2))) + 72p₂*(6.25p₂ - (60p₃)))

In [21]:
par_f = [-1,-0.5,-10]
P(flu,c,par_f)'[4,1]/P(flu,c,par_f)'[4,3]

-0.09313371727529571

Miramos primero el caso donde las variables están desacopladas, es decir con $\chi_1=0$.

In [22]:
P(flu,c,[-1,-0,-2])

5×5 Matrix{Float64}:
 0.821918  0.913242   -2.3032e-15    1.9843e-14  -1.73017e-18
 0.024726  0.821918   -0.271687      2.69735     -0.000190454
 0.0       0.0        -5.29874      63.8535       0.00194622
 0.0       0.0         2.62439     -26.0554       0.00183971
 0.0       0.0       -15.5246      167.936        0.955344

Vemos que se corresponde con la siguiente matriz racional.

In [23]:
M = [0 2//3 0 0 0;
    1//2 0 -1//4 0 -1//4;
    0 0 0 5//3 0;
    0 0 1//5 0 1//5;
    0 0 0 4//3 0];

In [24]:
P(flu,c,[-1,0,-1]) - M

5×5 Matrix{Float64}:
  0.821918  0.246575   -2.3032e-15    1.9843e-14  -1.73017e-18
 -0.475274  0.821918   -0.0216872     2.69735      0.24981
  0.0       0.0        -5.29874      62.1869       0.00194622
  0.0       0.0         2.42439     -26.0554      -0.19816
  0.0       0.0       -15.5246      166.602        0.955344

In [25]:
eigvals(M)

5-element Vector{Float64}:
 -0.7745966692414842
 -0.5773502691896256
  3.0357660829594124e-18
  0.5773502691896258
  0.7745966692414837

In [26]:
1/0.7745966692414837^2 - 5/3

-1.5543122344752192e-15

In [27]:
1/0.5773502691896258^2 - 3

-8.881784197001252e-16

In [28]:
eigvals(P(flu,c,[-1,-0.5,-1]))

5-element Vector{ComplexF64}:
 0.6920261648426755 + 0.0im
  0.816210922703936 - 0.25976511065367136im
  0.816210922703936 + 0.25976511065367136im
  1.028445706997088 + 0.0im
 38.838556655867706 + 0.0im

Hacemos lo mismo para la fuente.

In [29]:
function Is(flu, χ, ξ)
    #(χ, ξ) = par 
    μ = flu[1] 
    χ₀= χ[1]
    χ₁= -χ[2] # lo hacemos positivo
    κ = χ₀*ξ[1]/χ₁^2 # OK 
    λ = χ₀*ξ[2]/χ₁^2  # OK
    η = χ₀*ξ[3]/χ₁^2
    T = (abs(μ))^(-1//2) # use μ positive, so I changed μ -> -μ
    v = flu[2]
    x1 = flu[3]
    x2 = flu[4]
    x3 = flu[5]
    γ = (1. - v^2)^(-1//2)
    #Is[1] = 0.
    #Is[2] = 0.
    #Is[3] = -2//5*a*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*λ)
    #Is[4] = -2//5*a*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*λ)
    #Is[5] = -2//5*a*(γ^2*v^2+1//4)*T*x1/λ/γ - 2γ*v*x2/T/κ - T*x3/(γ*λ)
    #return Is[:]      #(1 - ℯ^(-5. *t))
    return [0.;
            0.;
            -(3//10*(γ^2-1//4)*x1/(γ*λ)   + 2γ*v*x2/κ      - v^2*x3/(γ*η))/μ^4/T;
            -(3//10*γ*v*x1/λ              + γ*(v^2+1)*x2/κ - v*x3/(γ*η))/μ^4/T;
            -(3//10*(γ^2*v^2+1//4)*x1/λ/γ + 2γ*v*x2/κ      - x3/(γ*η))/μ^4/T;
            #-2//5*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*η);
            #-2//5*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*η);
            #-2//5*(γ^2*v^2+1//4)*T*x1/η/γ - 2γ*v*x2/T/κ + T*x3/(γ*η)
            #-2//5*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*λ);
            #-2//5*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*λ);
            #-2//5*(γ^2*v^2+1//4)*T*x1/λ/γ - 2γ*v*x2/T/κ - T*x3/(γ*λ)
            ]
end

Is (generic function with 1 method)

La chequeamos con valores de Marcelo.

In [30]:
t=0
flu = [- 5.; 0.0; 2.1; 0.5; 5.1]
par = [-1., -2., -10.]
par_s = [1/15π, 1/1π, 1/1π]
I_c = [0.; 0.; - 0.09488575328013785; - 0.12155655385334033; - 0.12140081195218293]
Is(flu,par,par_s) - I_c;

In [31]:
@variables q[1:3]  
Is(f,p,q);

In [32]:
JSS = Symbolics.jacobian(Is(f,p,q),f);

In [33]:
JS_exp = Symbolics.build_function(JSS, f, p,q);

In [34]:
JSac = eval(JS_exp[1]);

In [35]:
flu=[1.;-0.9;0;0;0];
con=ones(5)
#par_f = [-1,-2,-10]
par_f = [-1,-0.0,1]
#JSac(flu,par_f,par_s)
#eigvals(JSac(flu,par_f,par_s)) #esta no es la matríz correcta para sacarle los autovalores
eigvals(P(flu,c,par_f))

5-element Vector{Float64}:
 -92.02982767323965
  -1.0274021719908326
  -0.9721870563644237
  -0.723651380445861
  -0.6716485600739354

Si $\chi_1=0$ los autovalores (velocidades) no dependen de los otros $\chi$ y hay uno que crece muchísimo cuando $v \to 1$. Eso trae problemas para evolucionar las ecuaciones por el CFL.

In [36]:
II(flu,c,p,q) = Jac(flu,c,p) \ JSac(flu,p,q)

II (generic function with 1 method)

In [37]:
II(flu,c,p,q);

In [38]:
par_f = [-1,-0.5,-1]
par_s = [-1/15π, -1/1π, -0.1/1π]
eigvals(1*im .* P(flu,c,par_f) - 0.0 .*II(flu,c,par_f,par_s))

5-element Vector{ComplexF64}:
     -0.3227243212134478 - 0.515560156372672im
 -1.4210854715202004e-14 + 34.57306291086245im
 -1.8051398841728837e-15 - 0.6874567395422221im
   6.249469941448383e-17 - 1.0387990014166986im
      0.3227243212134496 - 0.5155601563726537im

In [39]:
eigvals(-II(flu,c,par_f,par_s)) #notemos que esta tiene dos autovalor positivos!

5-element Vector{Float64}:
 -0.22275430636752863
 -0.0011447429759480713
 -0.0
  0.0
  0.011345525787119164

Vemos que un par de autovalores tiene parte real positiva. 

In [40]:
eigvecs(II(flu,c,par_f,par_s))

5×5 Matrix{Float64}:
  0.104016   1.0  0.0   0.107241     0.00723379
 -0.0124128  0.0  1.0   0.00259509  -0.00110536
  0.931304   0.0  0.0   0.990192    -0.00388408
 -0.112165   0.0  0.0   0.0230413    0.0127446
  0.330331   0.0  0.0  -0.0864993    0.999884

In [41]:
par_f = [-1,-1,-10]
E = eigvecs(10*im .* P(flu,c,par_f) - 0. * II(flu,c,par_f,par_s));

In [42]:
[E[:,1]; E[:,5]]

10-element Vector{ComplexF64}:
    0.03392902361428295 - 0.18565151104423314im
   -0.00393414662988325 + 0.03066398308213427im
   0.021551135746863882 - 0.00028413646656198083im
   0.040290450405110305 - 0.000425190158447504im
     0.9804787336140335 + 0.0im
   0.033929023614427796 + 0.18565151104368102im
 -0.0039341466299043625 - 0.030663983082049272im
   0.021551135746865062 + 0.0002841364665643071im
    0.04029045040511333 + 0.0004251901584501547im
     0.9804787336141354 + 0.0im

In [43]:
[E[:,2]; E[:,4]]

10-element Vector{ComplexF64}:
   -0.05255000756713335 + 4.2358477836401676e-14im
  0.0014752804718655042 + 8.193641078124703e-15im
    0.03879691380912619 - 6.570959054652548e-14im
    0.06947039060971613 - 1.1669484822895981e-13im
     0.9954421050767563 + 0.0im
   0.003806727693345729 - 3.159495234883458e-14im
 0.00036593454473584405 - 1.736078728692414e-15im
  -0.013924794939869825 + 3.536233805778721e-14im
   -0.02533848314073266 + 6.315260814293566e-14im
     0.9995746276662825 + 0.0im

In [44]:
E[:,3]

5-element Vector{ComplexF64}:
 -0.11979728242697228 - 5.3308865568396646e-17im
 -0.02345891319720768 + 3.5845587294854736e-17im
  -0.6035448754220478 - 5.366800753803247e-17im
 -0.19072302482856826 - 2.4062511965600164e-17im
      0.7644976139052 + 0.0im

In [45]:
Ei = inv(E)

5×5 Matrix{ComplexF64}:
    12.8109-2.0938im       …    0.0944203-0.00473467im
   -17.7013-3.35003e-11im        0.137714-3.78955e-13im
 -0.0527449-1.71738e-16im     -5.35308e-5+1.4299e-17im
   -7.46387+1.02343e-11im        0.678088+2.44954e-13im
    12.8109+2.0938im            0.0944203+0.00473467im

In [46]:
Ei[3,:]

5-element Vector{ComplexF64}:
  -0.05274485756154907 - 1.717376241217039e-16im
  -0.32278677681983486 - 2.3869795029440866e-15im
   -1.9716907670023622 + 1.457167719820518e-16im
    1.0688476231969106 + 3.8163916471489756e-16im
 -5.353076327775631e-5 + 1.429903030955879e-17im

In [47]:
Ei[5,:]'*E[:,5]

1.5595857938421887e-12 - 1.5918863449648768e-13im

In [48]:
ET = eigvecs(10*im .* P(flu,c,par_f)');

In [49]:
[ET[:,1]; ET[:,5]];

In [50]:
[ET[:,2]; ET[:,3]];

In [51]:
ET[:,4] - ET[1,4]/Ei[4,1] .* Ei[4,:];

In [52]:
1/(Ei[3,1]/Ei[3,3])

37.38166824512807 - 1.244776451324114e-13im

In [53]:
1/(Ei[3,1]/Ei[3,5])

0.0010149001391328084 - 2.744026151039468e-16im

 [-1,-1,-1] -> 16/5, 4  $\frac{4\chi_2}{\chi_1}(4/5,1)$
 [-1,-2,10] -> -16, 20
 [-1,-0.5,-1] -> -64, 80

In [54]:
4χ[2]/χ[1]*[4/5;1]

2-element Vector{Float64}:
 1.6
 2.0

In [55]:
function coc(par_f)
    return 4par_f[3]/par_f[2]*[-4/5 ; 1]
end

coc (generic function with 1 method)

In [56]:
coc([-1,-1,-10])

2-element Vector{Float64}:
 -32.0
  40.0

In [57]:
coc([-1,-0.5,-10])

2-element Vector{Float64}:
 -64.0
  80.0

Alternativamente podemos quedarnos con las expresiones en derivadas del los campos del fluido.
Es decir:

$(\zeta^a, \zeta^{ab})$ -> $f^j = (\mu, v, x_i)$ -> $(-1,0,0,0,0)$

$$
\partial_t c^j = \partial_i F^{ij}(f) + I^j(f) = \frac{\partial F^{ij}}{\partial f^k}\partial_i f^k + I^j(j)
$$

In [247]:
flu=[1.;0.5;0;0;0];
#con=ones(5)
par_f = [-1.,-0.,1.]
#par_f = [-1,-0.5,-10]
par_s = [-1/15π, -1/1π, -0.1/1π]
con = F(flu,zeros(5), par_f);
PP = inv(Jac(flu,con,par_f))*(JFac(flu,par_f) + 0.0.*JSac(flu,par_f,par_s))

5×5 Matrix{Float64}:
  0.363636  -0.727273   0.0        0.0       0.0
 -0.306818   0.363636   0.787663  -1.21179   0.0817958
  0.0        0.0       -1.08955    4.52239  -0.0727612
  0.0        0.0        0.970149  -1.49254   0.100746
  0.0        0.0       -1.65174    5.41294   0.574627

In [251]:
flu=[1.;-0.5;0;0;0];
#con=ones(5)
par_f = [1.,-0.,-1.]
#par_f = [-1,-0.5,-10]
par_s = [-1/15π, -1/1π, -0.1/1π]
con = F(flu,zeros(5), par_f);
PP = inv(Jac(flu,con,par_f))*(JFac(flu,par_f) + 0.0.*JSac(flu,par_f,par_s))

5×5 Matrix{Float64}:
 -0.363636  -0.727273   0.0      0.0       0.0
 -0.306818  -0.363636  -1.92674  1.21179   0.0817958
  0.0        0.0       -4.8806   4.52239   0.0727612
  0.0        0.0       -2.37313  1.49254   0.100746
  0.0        0.0       -5.35323  5.41294  -0.574627

In [225]:
eigvals(PP)

5-element Vector{ComplexF64}:
    -0.5075617914690567 - 1.61135948756819im
 -8.348972414653242e-17 - 0.7399675920277918im
                    0.0 - 0.8360138566096933im
  3.524671364951802e-17 + 0.10874112933696656im
     0.5075617914690568 - 1.6113594875681951im

In [226]:
Et = eigvecs(PP);

In [227]:
Ett = inv(Et);

In [228]:
H = adjoint(Ett)*Ett
#latexify(H)

5×5 Matrix{ComplexF64}:
   0.710938+0.0im          …   1.05163+2.39746e-15im
 1.7244e-16-8.74611e-17im      1.63708+3.62044e-15im
   0.402992+6.28309e-16im       2.4638+1.84815e-14im
   -3.07429+3.81222e-15im     -23.1611-3.8903e-14im
    1.05163-2.39746e-15im      7.98949+0.0im

In [230]:
H*PP + adjoint(PP)*H

5×5 Matrix{ComplexF64}:
  5.36693e-17+0.0im          …  -1.82348e-16+1.11022e-16im
 -4.31408e-32+4.44089e-16im     -2.02858e-16+8.88178e-16im
  7.17487e-16-7.21645e-15im       6.8121e-15-4.88727im
 -5.10703e-16+1.19904e-14im      6.16235e-15+5.86066im
 -1.82348e-16-1.11022e-16im     -4.55691e-15+0.0im

In [231]:
eigvals(H)

5-element Vector{Float64}:
   0.44869088332762785
   0.4997069250020913
   1.3879128490688213
  21.53495632225366
 314.5948796342527

In [61]:
eigvals(H*(1.0im.* JFac(flu,par_f) - 0.0.*JSac(flu,con,par_s)))

5-element Vector{ComplexF64}:
  -18.77378089891949 - 16.060972860964633im
 -18.773780898919433 + 16.060972860964554im
                 0.0 + 0.0im
  18.773780898919416 - 16.06097286096449im
   18.77378089891951 + 16.060972860964554im

In [62]:
H*JSac(flu,con,p)

5×5 Matrix{Num}:
 0.0  0.0   0.0           0.0                0.0
 0.0  0.0   0.0           0.0                0.0
 0.0  0.0  -0.135(p₂^-1)  0.0                0.0
 0.0  0.0   0.0           3.0(p₁^-1)         0.0
 0.0  0.0   0.075(p₂^-1)  0.0         -(p₃^-1)

In [89]:
1/1.333333

0.7500001875000468

In [64]:
200/27

7.407407407407407